# {Teen Pregnancy and Sex Education in the US}📝

![Banner](./assets/banner.jpeg)

## Topic
*What problem are you (or your stakeholder) trying to address?*
📝 <!-- Answer Below -->

I am addressing the problem of teen pregnancy in the United States and examining if and how it relates to a lack of sex education. 

## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*
📝 <!-- Answer Below -->

The specific question I am seeking to answer with this project is: "Is there a correlation between lack of sex education and increased teen pregnancy?"

## What would an answer look like?
*What is your hypothesized answer to your question?*
📝 <!-- Answer Below -->

An answer could be expressed with a visualization such as a table or a bar chart showcasing the states with the highest and lowest teen pregnancy numbers, their overall population, and  their policies surrounding sex education. 

The answer itself would be: "Yes/No, there is/isn't strong evidence to suggest increased teen pregnancy directly correlates to a lack of sex education." 

## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
📝 <!-- Answer Below -->

My datasets are:
- CDC Wonder (https://wonder.cdc.gov/controller/datarequest/D149;jsessionid=C8DCABB90CDF366055F96EA1D525 and https://wonder.cdc.gov/controller/datarequest/D192;jsessionid=70DEF061E23E358712E1506B71DF). This is a query based data source that allows you to view birth rates with different filters. I have imported the 2016-2024 and 2024-2025 natality datasets. 

- The United States Census (https://data.census.gov/table?q=teen+population+by+state&y=2024). This is united states census data filtered to focus on teens aged 15-19. 

- Parents Defending Education: Sex Education Laws By State (https://actionpde.org/sex-education-laws-by-state/). This is a website showcasing sex education laws by state. I have downloaded the website data as a PDF and extracted the raw text using pdfplumber. 

I will relate these datasets by state. When applicable, I will also use year, "Mandate Sex Ed" (Sex Education Laws By State), and age. 

## Approach and Analysis
*What is your approach to answering your project question?*
*How will you use the identified data to answer your project question?*
📝 <!-- Start Discussing the project here; you can add as many code cells as you need -->

My approach will be to:
- Calculate the average and raw teen pregnancy rate per state using census population numbers and CDC natality data. 

- Extract data from the Sex Education Laws by State site and convert it into a usable table, including whether sex education is required and how comprehensive it is. 

- Connect datasets by year and when applicable "Mandata Sex Ed" and/or age. 

- Explore visualizations to illustrate the data and results. 

In [3]:
# Start your code here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pdfplumber


# Importing my sources: 

#Census data: 
census = pd.read_csv("ACSST1Y2024.S0902-2025-09-28T050122.csv")
#Birth data from 2016-2024
BirthData20162024 = pd.read_csv("Natality, 2016-2024 expanded.csv")
#Birth data from 2023 to July 2025
BirthData2025 = pd.read_csv("Provisional Natality, 2023 through Last Month.csv")

#Using pdfplumber to extract text from PDE-Action_-Sex-Education-State-Laws_v2-1.pdf, which will later be used to create a dataframe
#First extracting the first page as a test
with pdfplumber.open("PDE-Action_-Sex-Education-State-Laws_v2-1.pdf") as pdf_v1:
    pdf_full_text_v1 = ""
    for page in pdf_v1.pages:
            pdf_full_text_v1 += page.extract_text() + "\n"

#Extracting the full text from the PDF, 12 pages, as pdf_full_text
with pdfplumber.open("PDE-Action_-Sex-Education-State-Laws_v2-2.pdf") as pdf_v2:
    pdf_full_text_v2 = ""
    for page in pdf_v2.pages:
            pdf_full_text_v2 += page.extract_text() + "\n"

with pdfplumber.open("PDE-Action_-Sex-Education-State-Laws_v2-3.pdf") as pdf_v3:
    pdf_full_text_v3 = ""
    for page in pdf_v3.pages:
            pdf_full_text_v3 += page.extract_text() + "\n"

In [ ]:
#Configuring the PDF into usable data. This is going to be done by splitting the raw text into lines,
#then parsing each line for relevant data points such as state, Q1 - Mandate SexEd.?, Q2 - Opt-Out/Opt-In Policy?, and Notify parents.
import re
import pandas as pd

#Using regular expressions to look for states that are abbreviayed with capital letters such as KY, CA, TX, etc. 
#Lines of text are formatted as: State Abbreviation, Q1 answer, Statute, Q2 answer, Statute, Notify Parents answer.
#The statues aren't needed for visualizations, so I'm skipping over them. 


#Function to get my states
state_pattern = re.compile(r'^[A-Z]{2}\b')


#The next step is to break down the answers of the questions into basic "Yes or No" For Q1 and
#"Yes or No" for Q2 and Notify Parents. 
def categorize_answer(answer):
    answer = answer.lower()
    if "yes" in answer:
        return "Yes"
    elif "no" in answer:
        return "No"
    elif "partial" in answer:
        return "Partial"
    else:
        return "Unknown"
    
#Using the first word of each line to extract the answer to Q1, Q2, and Notify Parents.
pdf_lines = pdf_full_text_v3.split('\n')
data = []
for line in pdf_lines:
    line = line.strip()
    state_match = state_pattern.match(line)
    parts = line.split()
    if len(parts) >= 6 and state_match:
        state = parts[0]
        q1 = categorize_answer(parts[1])
        q2 = categorize_answer(parts[3])
        notify = categorize_answer(parts[5])
        data.append({
            "State": state,
            "Mandate SexEd": q1,
            "Opt-Out/Opt-In Policy": q2,
            "Notify Parents": notify
    
        })
    
#Putting everything into a pandas datafame
StateLawsDF = pd.DataFrame(data)
# StateLawsDF["Mandate SexEd"] = StateLawsDF["Mandate SexEd"].apply(categorize_answer)
# StateLawsDF["Opt-Out/Opt-In Policy"] = StateLawsDF["Opt-Out/Opt-In Policy"].apply(categorize_answer)
# StateLawsDF["Notify Parents"] = StateLawsDF["Notify Parents"].apply(categorize_answer)
#Fixing any duplicates with state
StateLawsDF = StateLawsDF.drop_duplicates(subset=["State"])

In [ ]:
#Testing the dataframe
#Test for print: 
print(pdf_full_text_v2)
#print(StateLawsDF)
#Test for duplicates: print(StateLawsDF["State"].value_counts())

State Q1 - Mandate
AL No
AK Partial
AZ No
AR Partial
CA Yes
CO No
CT Partial
DC Yes
DE Yes
FL
Partial
GA Yes
HI Yes
ID No
IL
Partial
IN Partial
IA Yes
KS Yes
KY Yes
LA No
ME Yes
MD Yes
MA No
MI
Partial
MN Yes
MS
Yes
MO
Partial
MT Yes
NE No
NV
Partial
NH Yes
NJ Yes
NM Yes
NY Partial
NC Yes
ND Yes
OH Yes
OK Partial
OR Yes
PA
Partial
RI Yes
SC Yes
SD No
TN Yes
TX No
UT Yes
VT Yes
VA Yes
WA Yes
WV Yes
WI
Partial
WY No



## Resources and References
*What resources and references have you used for this project?*
📝 <!-- Answer Below -->

Resourced used: 
- https://wonder.cdc.gov/
- https://data.census.gov/
- https://actionpde.org/sex-education-laws-by-state/
- https://www.pdfplumber.com/
- https://www.cdc.gov/nchs/state-stats/births/teen-births.html

In [4]:
# ⚠️ Make sure you run this cell at the end of your notebook before every submission!
!jupyter nbconvert --to python source.ipynb

[NbConvertApp] Converting notebook source.ipynb to python
[NbConvertApp] Writing 4137 bytes to source.py
